In [6]:
import re
import os
import pandas as pd
import numpy as np

os.getcwd()
os.chdir('/Users/Bowen/Documents/2017Spring/ADS/Spr2017-proj4-team-4-master/data/nameset/')
with open('AKumar.txt') as f:
    content = f.readlines()

content=pd.DataFrame([x.lower().rstrip('\n').split('<>') for x in content])
content.columns=['author','title','publish']

author=content['author']
author_splited=[x.split(';') for x in author]

pattern='[0-9]+_[0-9]+'
# generate index
index=[re.search(pattern,x).group(0) for x in author]
index_splited=pd.DataFrame([x.split('_') for x in index])
# generate author
index_len=[len(x) for x in index]
author=pd.DataFrame([x[m:len(x)] for x,m in zip(author, index_len)])

#combine them together
content=pd.concat([index_splited,author,content[['title','publish']]],axis=1)
content.columns=['author','index','co-author','title','publish']

In [3]:
os.getcwd()
os.chdir('/Users/Bowen/Documents/2017Spring/ADS/Spr2017-proj4-team-4-master/data/')

dtm=pd.read_csv('pca.csv')
dtm.shape
dtm=dtm.iloc[:,1:]
c=dtm.shape[1]
r=dtm.shape[0]


lamb=[1/c]*c

def cor(x,y):
    x_y=pd.concat([x,y],axis=1)
    correlation=x_y.apply(np.var,1)
    return (correlation)    

def score(var,lamb):
    return (sum(var*lamb))

cor_list=[]
location_list=[]



''' loop first time  '''
for i in range(r):
    for j in range(i+1,r):
        var=cor(dtm.iloc[i,:],dtm.iloc[j,:])
        inter=score(var,lamb)
        cor_list.append(inter)
        location_list.append([i,j])

cor_list=cor_list/np.mean(cor_list)
location=np.array(np.where(cor_list==min(cor_list)))
location_list=np.array(location_list)
location_min=location_list[location]
''' group part'''
group=[x for x in range(r)]
for i in range(location.size):
    num1=location_min[0][i][0]
    num2=location_min[0][i][1]
    group[num2]=group[num1]



In [29]:
### Accuracy Function
label=[int(x) for x in content['author']]

A=0
B=0
C=0
D=0
for i in range(len(group)):
    for j in range(len(group)):
        if group[i]==group[j] and label[i]==label[j]:
            A+=1
        elif group[i]==group[j] and label[i]!=label[j]:
            B+=1
        elif group[i]!=group[j] and label[i]==label[j]:
            C+=1
        else:
            D+=1

Precision=A/(A+B)
Recall=A/(A+C)
F1=2*Precision*Recall/(Precision+Recall)
Accuracy=(A+D)/(A+B+C+D)

In [31]:
print(' '.join(['Precision:',str(Precision)]))
print(' '.join(['Recall:',str(Recall)]))
print(' '.join(['F1:',str(F1)]))
print(' '.join(['Accuracy:',str(Accuracy)]))

Precision: 1.0
Recall: 0.08075495049504951
F1: 0.14944174062410534
Accuracy: 0.8003896801934963
